In [ ]:
# !pip install kaggle
# ! mkdir ~/.kaggle

# import json
# kaggle_token = {"username":"uom200088d","key":"a41aca6e79872c8ff40d9efd56579ca3"}
# with open("kaggle.json", "w") as f:
#   token_json = json.dumps(kaggle_token)
#   f.write(token_json)

# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download -c amp-parkinsons-disease-progression-prediction
# ! unzip amp-parkinsons-disease-progression-prediction.zip 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import amp_pd_peptide_310
import numpy as np
import math

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# # Load the training datasets into a Pandas DataFrame
# train_proteins = pd.read_csv("train_proteins.csv")
# train_peptides = pd.read_csv("train_peptides.csv")
# train_clinical = pd.read_csv("train_clinical_data.csv")

train_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
train_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
train_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")

In [ ]:
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


We get all the protein npx values for a given patient visit.

In [ ]:
proteins = train_proteins.pivot(index = ['visit_month','patient_id'], columns = 'UniProt', values = 'NPX')
proteins.head()

UniProt                  O00391    O00533   O00584   O14498    O14773  \
visit_month patient_id                                                  
0           55          11254.3  732430.0  39585.8  41526.9  31238.00   
            1517        11648.9  419015.0  35053.8      NaN  17466.30   
            1923        21361.8  866985.0  32035.1      NaN  13373.10   
            2660            NaN  579829.0  28259.8  29883.2  19134.10   
            3636        10887.0  267402.0  11390.4  15696.3   6646.16   

UniProt                  O14791    O15240   O15394    O43505    O60888  ...  \
visit_month patient_id                                                  ...   
0           55          4202.71  177775.0  62898.2  333376.0  166850.0  ...   
            1517        2784.40  118752.0  40274.9  166322.0   93873.7  ...   
            1923            NaN  269106.0  81580.2  315740.0  297865.0  ...   
            2660        1884.61  134409.0  70664.8  213295.0  229999.0  ...   
            3636        2962.85   61555.3  25927.8   72635.7  110550.0  ...   

UniProt                   Q9HDC9   Q9NQ79    Q9NYU2    Q9UBR2   Q9UBX5  \
visit_month patient_id                                                   
0           55          365475.0  35528.0   97005.6  23122.50  60912.6   
            1517        306280.0  24931.3       NaN  12610.90  40018.9   
            1923         63839.7  58941.2  116485.0  14041.30  66752.8   
            2660        224909.0  32224.2  127529.0  14148.60  26387.0   
            3636        232382.0      NaN   72393.6   7752.62  16678.1   

UniProt                   Q9UHG2   Q9UKV8   Q9UNU6    Q9Y646    Q9Y6R7  
visit_month patient_id                                                  
0           55          408698.0      NaN  29758.8  23833.70  18953.50  
            1517        164826.0  55379.8  31563.0  19771.60  14699.50  
            1923        499634.0  64302.3  25588.8  26346.20  23597.10  
            2660        294153.0  55047.2  17880.5  25630.00  17990.90  
            3636        141754.0      NaN      NaN   7985.07   8587.01  

[5 rows x 227 columns]

In [ ]:
merged_df = pd.merge(train_clinical,proteins,on = ['visit_month','patient_id'],how = 'inner')

We get some peptide abundance values for a given patient visit.

In [ ]:
peptides = train_peptides.pivot(index = ['visit_month','patient_id'], columns = 'Peptide', values = 'PeptideAbundance')
peptides.head()

Peptide                 AADDTWEPFASGK  \
visit_month patient_id                  
0           55              8984260.0   
            1517            9465580.0   
            1923           12952600.0   
            2660            5831550.0   
            3636            3204710.0   

Peptide                 AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
visit_month patient_id                                                     
0           55                                                   53855.6   
            1517                                                 36200.7   
            1923                                                 55762.3   
            2660                                                 62855.0   
            3636                                                     NaN   

Peptide                 AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  \
visit_month patient_id                                                  
0           55                                 8579740.0          NaN   
            1517                               8639050.0      32892.8   
            1923                               6101600.0      22141.7   
            2660                               1787740.0      29919.2   
            3636                               6754210.0      20737.9   

Peptide                 AATGEC(UniMod_4)TATVGKR  AATVGSLAGQPLQER  \
visit_month patient_id                                             
0           55                         19735.40         114400.0   
            1517                       16584.90          34969.8   
            1923                            NaN         105439.0   
            2660                       10011.50          98348.3   
            3636                        9488.48          50767.4   

Peptide                 AAVYHHFISDGVR  ADDKETC(UniMod_4)FAEEGK  \
visit_month patient_id                                           
0           55                46371.1                  70412.0   
            1517              39739.1                 763708.0   
            1923              46003.8                 421481.0   
            2660                  NaN                 384980.0   
            3636              24156.1                 286174.0   

Peptide                 ADDKETC(UniMod_4)FAEEGKK  ADDLGKGGNEESTKTGNAGSR  ...  \
visit_month patient_id                                                   ...   
0           55                         7404060.0                39389.3  ...   
            1517                             NaN                60879.2  ...   
            1923                       4815540.0                41023.5  ...   
            2660                       5558470.0                36730.5  ...   
            3636                       4588830.0                20657.2  ...   

Peptide                 YSLTYIYTGLSK    YTTEIIK  YVGGQEHFAHLLILR  \
visit_month patient_id                                             
0           55              201158.0  16492.300        3810270.0   
            1517            210259.0   6789.370        3956620.0   
            1923            315900.0  17310.700        2811160.0   
            2660            290225.0   6626.350        6159900.0   
            3636            209499.0    258.249        4280990.0   

Peptide                 YVM(UniMod_35)LPVADQDQC(UniMod_4)IR  \
visit_month patient_id                                        
0           55                                     106894.0   
            1517                                   101069.0   
            1923                                    59059.2   
            2660                                    15559.7   
            3636                                        NaN   

Peptide                 YVMLPVADQDQC(UniMod_4)IR  YVNKEIQNAVNGVK  YWGVASFLQK  \
visit_month patient_id                                                         
0           55                          580667.0        131155.0    165851.0   
       

For a given patient visit, we have updrs scores and protein data in the merged_df dataframe.

In [ ]:
merged_df = pd.merge(merged_df,peptides,on = ['visit_month','patient_id'],how = 'inner')

In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,11254.3,732430.0,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,13163.6,630465.0,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,55_12,55,12,10.0,10.0,41.0,0.0,On,15257.6,815083.0,...,231772.0,17873.8,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,55_36,55,36,17.0,18.0,51.0,0.0,On,13530.8,753832.0,...,185290.0,18580.5,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,11218.7,399518.0,...,226314.0,6399.8,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1


In [ ]:
merged_df.shape

(1068, 1203)

In [ ]:
columns = list(merged_df.columns)[8:]

In [ ]:
print(columns)

['O00391', 'O00533', 'O00584', 'O14498', 'O14773', 'O14791', 'O15240', 'O15394', 'O43505', 'O60888', 'O75144', 'O75326', 'O94919', 'P00441', 'P00450', 'P00734', 'P00736', 'P00738', 'P00746', 'P00747', 'P00748', 'P00751', 'P01008', 'P01009', 'P01011', 'P01019', 'P01023', 'P01024', 'P01031', 'P01033', 'P01034', 'P01042', 'P01344', 'P01591', 'P01594', 'P01608', 'P01621', 'P01717', 'P01780', 'P01833', 'P01834', 'P01857', 'P01859', 'P01860', 'P01861', 'P01876', 'P01877', 'P02452', 'P02647', 'P02649', 'P02652', 'P02655', 'P02656', 'P02671', 'P02675', 'P02679', 'P02747', 'P02748', 'P02749', 'P02750', 'P02751', 'P02753', 'P02760', 'P02763', 'P02765', 'P02766', 'P02768', 'P02774', 'P02787', 'P02790', 'P02792', 'P04004', 'P04075', 'P04156', 'P04180', 'P04196', 'P04207', 'P04211', 'P04216', 'P04217', 'P04275', 'P04406', 'P04433', 'P05060', 'P05067', 'P05090', 'P05155', 'P05156', 'P05408', 'P05452', 'P05546', 'P06310', 'P06396', 'P06454', 'P06681', 'P06727', 'P07195', 'P07225', 'P07333', 'P07339',

We replaced outliers with median when the distribution is skewed, otherwise with mean.

In [ ]:
from scipy.stats import shapiro

# Function to replace outliers with median or mean based on distribution
def replace_outliers(data):
    data_copy = data.copy()  # Create a copy of the data
    median = data_copy.median()
    mean = data_copy.mean()
    is_skewed = shapiro(data_copy)[1] < 0.1  # Shapiro-Wilk test for skewness
    k = 1.5
    if is_skewed:
        data_copy[data_copy < median - k * data_copy.std()] = median
        data_copy[data_copy > median + k * data_copy.std()] = median
    else:
        data_copy[data_copy < mean - k * data_copy.std()] = mean
        data_copy[data_copy > mean + k * data_copy.std()] = mean
    
    return data_copy

# Replace outliers in each column based on distribution
for column in columns:
    merged_df[column] = replace_outliers(merged_df[column])

In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,11254.3,732430.0,...,201158.0,8996.806715,3810270.0,106894.0,580667.0,131155.000000,165851.000000,437305.0,46289.2,14898.4
1,55_6,55,6,8.0,10.0,34.0,NaN,NaN,13163.6,630465.0,...,171079.0,13198.800000,4119520.0,113385.0,514861.0,103512.000000,144607.000000,457891.0,40047.7,20703.9
2,55_12,55,12,10.0,10.0,41.0,0.0,On,15257.6,815083.0,...,231772.0,8996.806715,5474140.0,116286.0,711815.0,92668.876265,125754.068949,452253.0,54725.1,21841.1
3,55_36,55,36,17.0,18.0,51.0,0.0,On,13530.8,753832.0,...,185290.0,8996.806715,2659660.0,90936.9,679163.0,128593.000000,125754.068949,498621.0,52792.7,13973.7
4,942_6,942,6,8.0,2.0,21.0,NaN,NaN,11218.7,399518.0,...,226314.0,6399.800000,NaN,57571.4,480951.0,80001.200000,79661.900000,573300.0,48005.8,15674.1


We replaced NaN with 0 indication the absense of protein for the test for training and also 0 was common in updrs_4 and updrs_3 so we replaced NaN in those columns with 0 as well.

In [ ]:
# # Function to replace NaN values with median or mean based on distribution
# def replace_nan(data):
#     data_copy = data.copy()  # Create a copy of the data
#     median = data_copy.median()
#     mean = data_copy.mean()
#     is_skewed = shapiro(data_copy.dropna())[1] < 0.1  # Shapiro-Wilk test for skewness
    
#     if is_skewed:
#         filled_data = data_copy.fillna(median)
#         replacement_value = median
#     else:
#         filled_data = data_copy.fillna(mean)
#         replacement_value = mean
    
#     return filled_data, replacement_value

# # Dictionary to store the replacement values
# replacement_values = {}

# # Replace NaN values in each column based on distribution
# for column in columns:
#     filled_data, replacement_value = replace_nan(merged_df[column])
#     merged_df[column] = filled_data
#     replacement_values[column] = replacement_value

# Function to replace NaN values with 0
def replace_nan(data):
    filled_data = data.fillna(0)
    return filled_data

# Replace NaN values in each column with 0
for column in list(merged_df.columns):
    filled_data = replace_nan(merged_df[column])
    merged_df[column] = filled_data



In [ ]:
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,0.0,0,11254.3,732430.0,...,201158.0,8996.806715,3810270.0,106894.0,580667.0,131155.000000,165851.000000,437305.0,46289.2,14898.4
1,55_6,55,6,8.0,10.0,34.0,0.0,0,13163.6,630465.0,...,171079.0,13198.800000,4119520.0,113385.0,514861.0,103512.000000,144607.000000,457891.0,40047.7,20703.9
2,55_12,55,12,10.0,10.0,41.0,0.0,On,15257.6,815083.0,...,231772.0,8996.806715,5474140.0,116286.0,711815.0,92668.876265,125754.068949,452253.0,54725.1,21841.1
3,55_36,55,36,17.0,18.0,51.0,0.0,On,13530.8,753832.0,...,185290.0,8996.806715,2659660.0,90936.9,679163.0,128593.000000,125754.068949,498621.0,52792.7,13973.7
4,942_6,942,6,8.0,2.0,21.0,0.0,0,11218.7,399518.0,...,226314.0,6399.800000,0.0,57571.4,480951.0,80001.200000,79661.900000,573300.0,48005.8,15674.1


In [ ]:
features = [list(merged_df.columns)[2]] + list(merged_df.columns)[8:]
print(features)

['visit_month', 'O00391', 'O00533', 'O00584', 'O14498', 'O14773', 'O14791', 'O15240', 'O15394', 'O43505', 'O60888', 'O75144', 'O75326', 'O94919', 'P00441', 'P00450', 'P00734', 'P00736', 'P00738', 'P00746', 'P00747', 'P00748', 'P00751', 'P01008', 'P01009', 'P01011', 'P01019', 'P01023', 'P01024', 'P01031', 'P01033', 'P01034', 'P01042', 'P01344', 'P01591', 'P01594', 'P01608', 'P01621', 'P01717', 'P01780', 'P01833', 'P01834', 'P01857', 'P01859', 'P01860', 'P01861', 'P01876', 'P01877', 'P02452', 'P02647', 'P02649', 'P02652', 'P02655', 'P02656', 'P02671', 'P02675', 'P02679', 'P02747', 'P02748', 'P02749', 'P02750', 'P02751', 'P02753', 'P02760', 'P02763', 'P02765', 'P02766', 'P02768', 'P02774', 'P02787', 'P02790', 'P02792', 'P04004', 'P04075', 'P04156', 'P04180', 'P04196', 'P04207', 'P04211', 'P04216', 'P04217', 'P04275', 'P04406', 'P04433', 'P05060', 'P05067', 'P05090', 'P05155', 'P05156', 'P05408', 'P05452', 'P05546', 'P06310', 'P06396', 'P06454', 'P06681', 'P06727', 'P07195', 'P07225', 'P07

In [ ]:
col_y = list(merged_df.columns)[3:7]
print(col_y)

['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']


We calculated MI score for selected features agains target variables and selected features that have MI score higher than 0.2

In [ ]:
from sklearn.feature_selection import mutual_info_regression

updrs = []
# Assuming merged_df is your dataframe and features and col_y are your lists
for y_col in col_y:
    mi_scores = mutual_info_regression(merged_df[features], merged_df[y_col],random_state=42)
    
    print(f"MI scores for target variable '{y_col}':")
    temp = []
    for i, feature in enumerate(features):
        print(f"{feature}: {mi_scores[i]}")
        if mi_scores[i] >= np.percentile(mi_scores,90):
          temp.append(feature)
    updrs.append(temp)
    print("\n")

MI scores for target variable 'updrs_1':
visit_month: 0.0029855646001353264
O00391: 0.03729061016676294
O00533: 0.0002290847600692203
O00584: 0.0320388110358909
O14498: 0.0
O14773: 0.0
O14791: 0.020481811065400812
O15240: 0.01906841059884634
O15394: 0.0
O43505: 0.011945843352934027
O60888: 0.03967238543482843
O75144: 0.0
O75326: 0.0
O94919: 0.009942304369892518
P00441: 0.02027163892084971
P00450: 0.0
P00734: 0.0
P00736: 0.00016120113967232896
P00738: 0.0
P00746: 0.0
P00747: 0.022693929696744775
P00748: 0.00248760632245304
P00751: 0.0
P01008: 0.0
P01009: 0.05020900128965966
P01011: 0.028038950564683418
P01019: 0.0
P01023: 0.009297195336418262
P01024: 0.012338550724329878
P01031: 0.0
P01033: 0.031271560124449316
P01034: 0.0
P01042: 0.0
P01344: 0.0
P01591: 0.0
P01594: 0.01993237485953614
P01608: 0.0372519806723588
P01621: 0.0
P01717: 0.0
P01780: 0.02128238619844014
P01833: 0.007374980772188788
P01834: 0.0
P01857: 0.0037637936102159486
P01859: 0.014845408747572009
P01860: 0.054295773465184

In [ ]:
col_x_1 = [features[0]] + updrs[0]
col_x_2 = updrs[1]
col_x_3 = updrs[2]
col_x_4 = updrs[3]

print(len(col_x_1))
print(len(col_x_2))
print(len(col_x_3))
print(len(col_x_4))

print(col_x_1)
print(col_x_2)
print(col_x_3)
print(col_x_4)

121
120
120
120
['visit_month', 'O00391', 'O60888', 'P01009', 'P01608', 'P01860', 'P02763', 'P02790', 'P04156', 'P04433', 'P08571', 'P09486', 'P13521', 'P13987', 'P14174', 'P14618', 'P20774', 'P35542', 'P40925', 'P51884', 'Q06481', 'Q12805', 'Q14118', 'Q14515', 'Q8NBJ4', 'Q9UBX5', 'Q9Y646', 'ADDKETC(UniMod_4)FAEEGK', 'AIGAVPLIQGEYMIPC(UniMod_4)EK', 'AKAYLEEEC(UniMod_4)PATLRK', 'ALEYIENLR', 'APLIPMEHC(UniMod_4)TTR', 'ASGSPEPAISWFR', 'ATLGPAVRPLPWQR', 'AVISPGFDVFAK', 'AYQGVAAPFPK', 'C(UniMod_4)NLLAEK', 'C(UniMod_4)PFPSRPDNGFVNYPAKPTLYYK', 'DGGFC(UniMod_4)EVC(UniMod_4)KK', 'DKETC(UniMod_4)FAEEGKK', 'DLLFKDSAHGFLK', 'DLLFRDDTVC(UniMod_4)LAK', 'EAEEETTNDNGVLVLEPARK', 'EALQGVGDMGR', 'EILSVDC(UniMod_4)STNNPSQAK', 'ELLESYIDGR', 'ETYGEMADC(UniMod_4)C(UniMod_4)AK', 'FEHC(UniMod_4)NFNDVTTR', 'FIYGGC(UniMod_4)GGNR', 'FMQAVTGWK', 'FNKPFVFLM(UniMod_35)IEQNTK', 'FVEGLPINDFSR', 'GAQTQTEEEMTR', 'GATLALTQVTPQDER', 'GDKVWVYPPEKK', 'GINVALANGK', 'GNPEPTFSWTK', 'GNQWVGYDDQESVK', 'GVALADFNR', 'HTSVQTTSSGSGP

Scale all the features except visit month

In [ ]:
# merged_df[features[1:]] = np.log(merged_df[features[1:]])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Apply Min-Max scaling to the selected columns
merged_df[features[1:]] = scaler.fit_transform(merged_df[features[1:]])

merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,O00391,O00533,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,0.0,0,0.620386,1.570409,...,-0.279848,0.408405,0.260469,1.475181,0.235654,2.008359,1.491925,-0.210313,0.278172,-0.078781
1,55_6,55,6,8.0,10.0,34.0,0.0,0,0.968402,0.925112,...,-1.084162,1.689448,0.476174,1.647191,0.019060,0.703791,0.794341,0.013068,-0.143541,0.491897
2,55_12,55,12,10.0,10.0,41.0,0.0,On,1.350084,2.093488,...,0.538772,0.408405,1.421039,1.724067,0.667314,0.192067,0.175273,-0.048111,0.848153,0.603683
3,55_36,55,36,17.0,18.0,51.0,0.0,On,1.035333,1.705854,...,-0.704159,0.408405,-0.542096,1.052321,0.559843,1.887449,0.175273,0.455033,0.717588,-0.169679
4,942_6,942,6,8.0,2.0,21.0,0.0,0,0.613897,-0.536462,...,0.392825,-0.383333,-2.397242,0.168142,-0.092552,-0.405764,-1.338243,1.265381,0.394156,-0.002530


In [ ]:
merged_df = merged_df.drop(columns = 'upd23b_clinical_state_on_medication')
merged_df.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,O00391,O00533,O00584,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,55_0,55,0,10.0,6.0,15.0,0.0,0.620386,1.570409,1.977435,...,-0.279848,0.408405,0.260469,1.475181,0.235654,2.008359,1.491925,-0.210313,0.278172,-0.078781
1,55_6,55,6,8.0,10.0,34.0,0.0,0.968402,0.925112,1.343447,...,-1.084162,1.689448,0.476174,1.647191,0.019060,0.703791,0.794341,0.013068,-0.143541,0.491897
2,55_12,55,12,10.0,10.0,41.0,0.0,1.350084,2.093488,0.075091,...,0.538772,0.408405,1.421039,1.724067,0.667314,0.192067,0.175273,-0.048111,0.848153,0.603683
3,55_36,55,36,17.0,18.0,51.0,0.0,1.035333,1.705854,0.075091,...,-0.704159,0.408405,-0.542096,1.052321,0.559843,1.887449,0.175273,0.455033,0.717588,-0.169679
4,942_6,942,6,8.0,2.0,21.0,0.0,0.613897,-0.536462,-0.782890,...,0.392825,-0.383333,-2.397242,0.168142,-0.092552,-0.405764,-1.338243,1.265381,0.394156,-0.002530


Train using XGBoost Regressor and testing with various metrics.

In [ ]:
x_1 = merged_df[col_x_1]
y_1 = merged_df['updrs_1']

x_2 = merged_df[col_x_2]
y_2 = merged_df['updrs_2']

x_3 = merged_df[col_x_3]
y_3 = merged_df['updrs_3']

x_4 = merged_df[col_x_4]
y_4 = merged_df['updrs_4']

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.1

x_1_train, x_1_val, y_1_train, y_1_val = train_test_split(x_1,y_1, test_size=test_size, random_state=0)
x_2_train, x_2_val, y_2_train, y_2_val = train_test_split(x_2,y_2, test_size=test_size, random_state=0)
x_3_train, x_3_val, y_3_train, y_3_val = train_test_split(x_3,y_3, test_size=test_size, random_state=0)
x_4_train, x_4_val, y_4_train, y_4_val = train_test_split(x_4,y_3, test_size=test_size, random_state=0)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
from xgboost import XGBRegressor

def grid_search_hyperparameters(x_train,y_train, x_val, y_val, n_est_range, depth_range):
    least_err_n_estimate = 1000
    best_n_estimate = 0
    best_depth = 0
    for n_estimate in n_est_range:
        least_err_depth = 1000
        for depth in depth_range:
            model = XGBRegressor(n_estimators = n_estimate, learning_rate=0.05, max_depth=depth, random_state=0)
            model.fit(x_train,y_train, early_stopping_rounds=5,
                      eval_set=[(x_val, y_val)], verbose=False)
            y_val_pred = model.predict(x_val)
            err = smape(y_val, y_val_pred)
            if err < least_err_depth:
                best_depth = depth
                least_err_depth = err
        if least_err_depth < least_err_n_estimate:
            best_n_estimate = n_estimate
            least_err_n_estimate = least_err_depth
    return best_n_estimate, best_depth, least_err_n_estimate

In [ ]:
n_est_1, depth_1, least_err_1 = grid_search_hyperparameters(x_1_train, y_1_train, x_1_val, y_1_val, range(80,120,5), range(3,15))
n_est_1, depth_1, least_err_1

(80, 8, 63.52282741598283)

In [ ]:
n_est_2, depth_2, least_err_2 = grid_search_hyperparameters(x_2_train, y_2_train, x_2_val, y_2_val, range(75,125,5), range(3,12))
n_est_2, depth_2, least_err_2

(75, 6, 91.68842563116213)

In [ ]:
n_est_3, depth_3, least_err_3 = grid_search_hyperparameters(x_3_train, y_3_train, x_3_val, y_3_val, range(75,125,5), range(3,10))
n_est_3, depth_3, least_err_3

(95, 7, 79.5597220809467)

In [ ]:
n_est_4, depth_4, least_err_4 = grid_search_hyperparameters(x_4_train, y_4_train, x_4_val, y_4_val, range(50,125,5), range(20,30))
n_est_4, depth_4, least_err_4

(60, 21, 80.47464293183495)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# model1 = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)
# model2 = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)
# model3 = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)
# model4 = RandomForestRegressor(n_estimators=1000, max_depth=100, random_state=42)

from xgboost import XGBRegressor

model1 = XGBRegressor(n_estimators = n_est_1, learning_rate=0.05, max_depth=depth_1, random_state=0)
model2 = XGBRegressor(n_estimators = n_est_2, learning_rate=0.05, max_depth=depth_2, random_state=0)
model3 = XGBRegressor(n_estimators = n_est_3, learning_rate=0.05, max_depth=depth_3, random_state=0)
model4 = XGBRegressor(n_estimators = n_est_4, learning_rate=0.05, max_depth=depth_4, random_state=0)

model1.fit(x_1_train,y_1_train, early_stopping_rounds=5,
          eval_set=[(x_1_val, y_1_val)], verbose=False)
model2.fit(x_2_train,y_2_train, early_stopping_rounds=5,
          eval_set=[(x_2_val, y_2_val)], verbose=False)
model3.fit(x_3_train,y_3_train, early_stopping_rounds=5,
          eval_set=[(x_3_val, y_3_val)], verbose=False)
model4.fit(x_4_train,y_4_train, early_stopping_rounds=5,
          eval_set=[(x_4_val, y_4_val)], verbose=False)

y_1_train_pred = model1.predict(x_1_train)
y_1_val_pred = model1.predict(x_1_val)
y_2_train_pred = model2.predict(x_2_train)
y_2_val_pred = model2.predict(x_2_val)
y_3_train_pred = model3.predict(x_3_train)
y_3_val_pred = model3.predict(x_3_val)
y_4_train_pred = model4.predict(x_4_train)
y_4_val_pred = model4.predict(x_4_val)

In [ ]:
def evaluate_model(y_train, y_train_pred, y_val, y_val_pred):
    temp_df = pd.DataFrame(columns = ['train_error','val_error'], index=['smape','rmse','mae'])
    
    temp_df.loc['smape','train_error'] = smape(y_train, y_train_pred)
    temp_df.loc['smape','val_error'] = smape(y_val, y_val_pred)
    
    temp_df.loc['rmse','train_error'] = (mean_squared_error(y_train, y_train_pred))**(0.5)
    temp_df.loc['rmse','val_error'] = (mean_squared_error(y_val, y_val_pred))**(0.5)
    
    temp_df.loc['mae','train_error'] = mean_absolute_error(y_train, y_train_pred)
    temp_df.loc['mae','val_error'] = mean_absolute_error(y_val, y_val_pred)

    return temp_df

In [ ]:
# for updrs_1
evaluate_model(y_1_train, y_1_train_pred, y_1_val, y_1_val_pred)

,train_error,val_error
smape,30.510845,63.522827
rmse,1.442511,4.123099
mae,1.059578,3.230059


In [ ]:
# for updrs_2
evaluate_model(y_2_train, y_2_train_pred, y_2_val, y_2_val_pred)

,train_error,val_error
smape,66.981296,91.688426
rmse,1.994201,4.748287
mae,1.537037,3.895943


In [ ]:
# for updrs_3
evaluate_model(y_3_train, y_3_train_pred, y_3_val, y_3_val_pred)

,train_error,val_error
smape,48.982392,79.559722
rmse,2.161865,12.355301
mae,1.657365,9.986903


In [ ]:
# for updrs_4
evaluate_model(y_4_train, y_4_train_pred, y_4_val, y_4_val_pred)

,train_error,val_error
smape,42.995367,80.474643
rmse,2.050639,12.753832
mae,1.279522,9.940342


In [ ]:
# for overall data and predictions
comb_y_train = pd.Series(y_1_train.append(y_2_train.append(y_3_train.append(y_4_train))))
comb_y_val = pd.Series(y_1_val.append(y_2_val.append(y_3_val.append(y_4_val))))
comb_y_train_pred = np.concatenate((y_1_train_pred, y_2_train_pred, y_3_train_pred, y_4_train_pred))
comb_y_val_pred = np.concatenate((y_1_val_pred, y_2_val_pred, y_3_val_pred, y_4_val_pred))

evaluate_model(comb_y_train, comb_y_train_pred, comb_y_val, comb_y_val_pred)

,train_error,val_error
smape,47.367475,78.811405
rmse,1.932386,9.418862
mae,1.383376,6.763312


In [ ]:
model = {"updrs_1":model1,"updrs_2":model2,"updrs_3":model3,"updrs_4":model4}
features_selected = {"updrs_1":col_x_1,"updrs_2":col_x_2,"updrs_3":col_x_3,"updrs_4":col_x_4}
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

In [ ]:
def get_predictions(my_train, pro, model):
    proteins = pro.pivot(index=['visit_month', 'patient_id'], columns='UniProt', values='NPX')
    my_train = pd.merge(my_train, proteins, on=['visit_month', 'patient_id'], how='left')

    # Check which columns from features are not in my_train
    new_columns = [col for col in features if col not in my_train.columns]

    # Add the new columns to my_train with default value 0
    my_train[new_columns] = 0

    for column in list(my_train.columns):
      filled_data = replace_nan(my_train[column])
      my_train[column] = filled_data

    my_train[features[1:]] = scaler.transform(my_train[features[1:]])

    for u in target:
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0

        # Predict    
        X = my_train[features_selected[u]]
        if u == "updrs_4":
          my_train['result_' + str(u)] = 0
        else:
          my_train['result_' + str(u)] = np.floor(model[u].predict(X.values))

    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp[['prediction_id', 'rating']]

            result = result.append(temp)
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

# Run once to check results
pred = get_predictions(train_clinical, train_proteins, model)


In [ ]:
def test_predictions(my_train, pro):
    proteins = pro.pivot(index=['visit_month', 'patient_id'], columns='UniProt', values='NPX')
    my_train = pd.merge(my_train, proteins, on=['visit_month', 'patient_id'], how='left')

    # Check which columns from features are not in my_train
    new_columns = [col for col in features if col not in my_train.columns]

    # Add the new columns to my_train with default value 0
    my_train[new_columns] = 0

    for column in list(my_train.columns):
      filled_data = replace_nan(my_train[column])
      my_train[column] = filled_data

    my_train[features[1:]] = scaler.transform(my_train[features[1:]])

    for u in target:
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0

        # Predict    
        X = my_train[features_selected[u]]
        my_train['result_' + str(u)] = np.floor(my_train[u])

    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp[['prediction_id', 'rating']]

            result = result.append(temp)
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

test = test_predictions(train_clinical, train_proteins)

In [ ]:
smape(pred['rating'],test['rating'])

75.1530364974374

In [ ]:
env = amp_pd_peptide_310.make_env()   # initialize the environment
iter_test = env.iter_test()  


In [ ]:
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
        
    result = get_predictions(test, test_proteins, model)

    env.predict(result)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
